In [1]:
from llm import LLM, LLM_Stream
from db import DB, Corpus, Doc, Line, Form, Predict, Lemma, LemmaRaw
from datetime import datetime

In [2]:
llm = LLM()
# llm.complete()
# llm.debug()

In [3]:
llm.lemmatize(llm.db.get_test_line().line)

[{'word_form': 'Apollonius',
  'lemma': 'Apollonius',
  'translation_en': 'Apollonius',
  'translation_ru': 'Аполлоний',
  'morph_analysis': '',
  'syntax_analysis': 'NOUN, SUBJECT'},
 {'word_form': 'cwæð',
  'lemma': 'cweþan',
  'translation_en': 'said',
  'translation_ru': 'сказал',
  'morph_analysis': 'VERB, PAST, 3SG',
  'syntax_analysis': 'VERB, VERB'}]

In [4]:
# llm_stream = LLM_Stream()
# llm_stream.stream_complete("Напиши короткое стихотворение")
# # llm_stream.debug()

In [5]:
# line: Line = db.tbl(Line, 1).one()
# line.forms

In [6]:
db = llm.db
line = llm.db.get_test_line()
raw = llm.response.raw
raw["created_at"]

'2025-01-11T10:38:05.196876Z'

In [7]:
with db.s as s:
  # form = Form(num=0, form="Mæg", lemma="mag") # type: ignore
  # corpus = Corpus(corpus="iswoc", path="", docs=[
  #   Doc(doc="forms.txt", 
  #     lines=[Line(num=0, line="Mæg gehyran se ðe", lemmas="mag gehyran se þe", 
  #       forms=[form],  
  # dict(exclude={"id"})
  model = llm.llm
  raw = llm.response.raw # model_dump_json()["raw"]
  usage = raw["usage"]
  prediction = Predict(
    line = line,
    test = True,
    model = model.model,
    temperature = model.temperature, 
    done = raw["done"], 
    at = datetime.fromisoformat(raw["created_at"]),
    load_duration = raw["load_duration"],
    prompt_eval_duration = raw["prompt_eval_duration"],
    eval_duration = raw["eval_duration"],
    prompt_tokens = usage["prompt_tokens"],
    completion_tokens = usage["completion_tokens"],
    no_eq = 0, 
    # raw=PredictRaw(promt="", content=""), # type: ignore
    # lemmas=[Lemma(form=form, lemma="mag", eq=True,
    #   raw=LemmaRaw(id=1, en="can", ru="может", morph="pronoun", syntax="subject", 
    #     raw={"word_form": "Mæg",
    #       "lemma": "mag",
    #       "translation_en": "can",
    #       "translation_ru": "может",
    #       "morph_analysis": "pronoun, 3rd person singular present subjunctive of magan (can)",
    #       "syntax_analysis": "subject"}) 
    # )] # type: ignore
  )
  s.add(prediction)
  s.commit()
db.stat()

1: corpus
5: doc
2541: line
28300: form
2: predict
0: predictraw
0: lemma
0: lemmaraw


In [9]:
db.df(Predict)

,id,line_id,model,done,no_eq,at,load_duration,prompt_eval_duration,eval_duration,prompt_tokens,completion_tokens,temperature,test
0,1,244,mistral:7b,True,0,2025-01-11 09:09:55.122290,27539000,500000000,3186000000,133,172,0.0,True
1,2,244,mistral:7b,True,0,2025-01-11 10:38:05.196876,299517042,581000000,3175000000,133,172,0.0,True


In [11]:
db.del_test()
db.df(Predict)

,id,line_id,model,done,no_eq,at,load_duration,prompt_eval_duration,eval_duration,prompt_tokens,completion_tokens,temperature,test
